In [9]:
# Import necessary libraries
import tensorflow as tf
import numpy as np
import pathlib
import datetime

# Print TensorFlow and NumPy versions
print(f"TensorFlow Version: {tf.__version__}")
print(f"NumPy Version: {np.version.version}")

TensorFlow Version: 2.12.0
NumPy Version: 1.23.5


In [10]:


# Define the directory containing the dataset
data_dir = pathlib.Path("./alexnetnew/80% of training image/")

# Count the total number of images in the dataset
image_count = len(list(data_dir.glob('*/*.jpg')))
print("Total number of images:", image_count)

# Get class names from the subdirectories
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
print("Class names:", CLASS_NAMES)

# Determine the number of output units (classes)
output_class_units = len(CLASS_NAMES)
print("Number of output units:", output_class_units)


Total number of images: 4800
Class names: ['new glioma after deleting' 'new meningioma after deleting'
 'new notumor after deleting' 'new pituitary after deleting']
Number of output units: 4


In [11]:

# Define the model architecture (AlexNet) (changed)
model = tf.keras.models.Sequential([
    # 1st Convolutional Layer
    tf.keras.layers.Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(227, 227, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, strides=(2, 2)),
    
    # 2nd Convolutional Layer
    tf.keras.layers.Conv2D(256, (5, 5), strides=(1, 1), activation='relu', padding="same"),  
    # changed filter size
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, strides=(2, 2)),
    
    # 3rd Convolutional Layer
    tf.keras.layers.Conv2D(384, (3, 3), strides=(1, 1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    
    # 4th Convolutional Layer
    tf.keras.layers.Conv2D(384, (3, 3), strides=(1, 1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    
    # 5th Convolutional Layer
    tf.keras.layers.Conv2D(256, (3, 3), strides=(1, 1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, strides=(2, 2)),
    
    # Flatten layer
    tf.keras.layers.Flatten(),
    
    # Fully connected Layer 1
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Add dropout for regularization
    
    # Fully connected Layer 2
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Add dropout for regularization
    
    # Output Layer
    tf.keras.layers.Dense(output_class_units, activation='softmax')
])


In [12]:

# Define input shape parameters
BATCH_SIZE = 32        # Batch size for training
IMG_HEIGHT = 227       # Height of the input images
IMG_WIDTH = 227        # Width of the input images

# Calculate steps per epoch
STEPS_PER_EPOCH = np.ceil(image_count / BATCH_SIZE)

# Preprocess the images with data augmentation
train_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Generate training data batches
train_data_gen = train_image_generator.flow_from_directory(
    directory=str(data_dir),
    batch_size=BATCH_SIZE,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    classes=list(CLASS_NAMES)
)

# Compile the model with Adam optimizer and learning rate scheduling
initial_learning_rate = 0.0001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True
)
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)  # changed optimizer
model.compile(optimizer=adam_optimizer, loss="categorical_crossentropy", metrics=['accuracy'])

# Print model summary
model.summary()


Found 4800 images belonging to 4 classes.
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 55, 55, 96)        34944     
                                                                 
 batch_normalization_15 (Bat  (None, 55, 55, 96)       384       
 chNormalization)                                                
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 27, 27, 96)       0         
 2D)                                                             
                                                                 
 conv2d_16 (Conv2D)          (None, 27, 27, 256)       614656    
                                                                 
 batch_normalization_16 (Bat  (None, 27, 27, 256)      1024      
 chNormalization)                                                
            

In [13]:

# Define custom callback to stop training
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get("accuracy") == 1.0 and logs.get("loss") < 0.03:
            print("\nReached 100% accuracy so stopping training")
            self.model.stop_training = True

callbacks = myCallback()

# Setup TensorBoard callback
log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)


In [14]:

# Train the model
history = model.fit(
    train_data_gen,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=10,
    callbacks=[tensorboard_callback, callbacks]
)

# Save the entire model
model.save('AlexNet_saved_model.h5')

Epoch 1/10
150/150 [==============================] - 812s 5s/step - loss: 1.6479 - accuracy: 0.5908
Epoch 2/10
150/150 [==============================] - 688s 5s/step - loss: 1.2150 - accuracy: 0.6469
Epoch 3/10
150/150 [==============================] - 651s 4s/step - loss: 0.9919 - accuracy: 0.6829
Epoch 4/10
150/150 [==============================] - 654s 4s/step - loss: 0.8361 - accuracy: 0.7021
Epoch 5/10
150/150 [==============================] - 642s 4s/step - loss: 0.6893 - accuracy: 0.7542
Epoch 6/10
150/150 [==============================] - 647s 4s/step - loss: 0.6458 - accuracy: 0.7660
Epoch 7/10
150/150 [==============================] - 797s 5s/step - loss: 0.5874 - accuracy: 0.7777
Epoch 8/10
150/150 [==============================] - 633s 4s/step - loss: 0.5383 - accuracy: 0.8102
Epoch 9/10
150/150 [==============================] - 658s 4s/step - loss: 0.4859 - accuracy: 0.8296
Epoch 10/10
150/150 [==============================] - 665s 4s/step - loss: 0.4909 - accura